# Random Forest and Boosting Lab

In this lab we will practice using Random Forest Regressor and Boosted Trees Regressor on the Project 6 Data.

> Instructor Notes:
- This walks the students through a sample dataset, they should actually do it on the full dataset they have created as part of Project 6.
- The code for this lab is shorter than usual in order to give the students time to practice with Tableau.

## 1. Load and inspect the data

As part of your work of project 6 you should have retrieved the top 250 movies from IMDB. Conduct this lab on the data you have retrieved.

In the [asset folder](../../assets/datasets/imdb_p6_sample.csv) you can find a subset of the movies, in case you have not completed yet Project 6.

1. Load the dataset and inspect it
- Assign the rating to a y vector and the binary columns to an X feature matrix
- What would you do with the year variable?
> Answer: normalize it and use it as feature

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [43]:
df = pd.read_csv('../../assets/datasets/imdb_p6_sample.csv')
df

,HA,rating,tconst,title,year,excellent,great,love,beautiful,best,hope,groundbreaking,amazing
0,1633889,9.3,tt0111161,The Shawshank Redemption,1994,0,1,0,1,0,0,0,1
1,1118799,9.2,tt0068646,The Godfather,1972,1,1,0,0,1,0,1,0
2,762879,9.0,tt0071562,The Godfather: Part II,1974,1,1,0,0,1,0,0,1
3,1616346,9.0,tt0468569,The Dark Knight,2008,1,1,1,0,1,0,1,1
4,835155,8.9,tt0108052,Schindler's List,1993,1,1,1,1,1,1,1,1
5,429172,8.9,tt0050083,12 Angry Men,1957,1,0,0,0,0,0,0,0
6,1280264,8.9,tt0110912,Pulp Fiction,1994,0,0,0,0,0,0,1,0
7,1175740,8.9,tt0167260,The Lord of the Rings: The Return of the King,2003,1,0,0,1,0,0,0,1
8,487214,8.9,tt0060196,"The Good, the Bad and the Ugly",1966,0,0,0,0,0,1,1,0
9,1298873,8.9,tt0137523,Fight Club,1999,0,1,0,0,0,1,0,0


In [44]:
y = df['rating']
X = df[list(df.columns)[-9:]]

In [45]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=77)

# # make df of just age and fare 
from sklearn.preprocessing import StandardScaler
##X =  StandardScaler().fit_transform(X)
scaler = StandardScaler().fit(X[['year']])
X_norm = scaler.transform(X[['year']])

X['year'] = X_norm[:,0]


/Applications/anaconda/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## 2. Decision Tree Regressor


1. Train a decision tree regressor on the data and estimate the rating
- Evaluate the score with a 3-fold shuffled cross validation
- Do a scatter plot of the predicted vs actual scores for each of the 3 folds, do they match?
    - They should align to a diagonal line.
- Add some text to the plot indicating the average $R^2$ coefficient

In [46]:
X.head()

,year,excellent,great,love,beautiful,best,hope,groundbreaking,amazing
0,0.401446,0,1,0,1,0,0,0,1
1,-0.903253,1,1,0,0,1,0,1,0
2,-0.784644,1,1,0,0,1,0,0,1
3,1.231709,1,1,1,0,1,0,1,1
4,0.342141,1,1,1,1,1,1,1,1


In [53]:
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

y = LabelEncoder().fit_transform(df['rating'])

cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=21)

dt = DecisionTreeClassifier(class_weight='balanced', random_state=21)
s = cross_val_score(dt, X, y, cv=cv, n_jobs=-1)


## 3. Random Forest Regressor


1. Train a random forest regressor on the data and estimate the rating
- Evaluate the score with a 3-fold shuffled cross validation
- Do a scatter plot of the predicted vs actual scores for each of the 3 folds, do they match?
- How does this plot compare with the previous one?
> Answer: points are tighter now, indicating a better fit

## 4. AdaBoost Regressor


1. Train a AdaBoost regressor on the data and estimate the rating
- Evaluate the score with a 3-fold shuffled cross validation
- Do a scatter plot of the predicted vs actual scores for each of the 3 folds, do they match?
- Compare with previous score

## 4. Gradient Boosting Trees Regressor


1. Train a Gradient Boosting Trees regressor on the data and estimate the rating
- Evaluate the score with a 3-fold shuffled cross validation
- Do a scatter plot of the predicted vs actual scores for each of the 3 folds, do they match?
- Compare with previous score

## 5. Tableau Practice

Practice using Tableau to inspect the data and also to plot the results.


## Bonus

Take the best model and try to improve it using grid search.